In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import numpy as np
import random
from sklearn.cross_validation import cross_val_score # K折交叉验证模块
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import svm
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.preprocessing import Normalizer
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.multiclass import OutputCodeClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.pipeline import Pipeline as ImPineline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.grid_search import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import lfilter

import time
%matplotlib inline

/Users/cql/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/cql/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# spec = total_train_feature.iloc[:,:-1].head()

def specLineNum(spec, windowSize=2, threshold=0.8):
    fs = pd.Series()
    ss = pd.Series()
    for i in range(windowSize, spec.shape[1]):
        s = spec.iloc[:,i-windowSize]
        e = spec.iloc[:,i]
        isFs = (e>threshold) & ((e-s)>threshold)
        isSS = (e<threshold) & ((e-s)<-threshold)
        if(fs.size==0):
            fs = isFs.astype(int)
        else:
            fs = fs + isFs.astype(int)
        if(ss.size==0):
            ss = isSS.astype(int)
        else:
            ss = ss + isSS.astype(int)
    combine = pd.concat([fs, ss], axis=1)
    combine.columns = ['fs','ss']
    return combine

In [3]:
def qsoFeature(data):
    std1 = data.iloc[:,850:880].std(axis=1)
    std2 = data.iloc[:,885:898].std(axis=1)
    std3 = data.iloc[:,900:1000].std(axis=1)
    f1 = (std1<0.3) & (std2>0.35) & (std3<0.35)
    f3 = (std2*std2)/(std1*std3+0.1)
    d1 = std1+0.2
    f2 = (std2>d1)
    qsof = pd.concat([f1, f2, f3], axis=1)
    qsof.columns = ['qso_f1','qso_f2','qso_f3']
    return (qsof)

In [4]:
def intervalEWFeature(d, start=0, end=-1, filterThrehold=1):
    cp = d
#     cp[abs(cp)<filterThrehold] = 0
    rd = cp.iloc[:, start:end]
    specLines = specLineNum(rd[abs(rd)>filterThrehold])
    ew_absorb = rd[(rd<0)&(abs(rd)>filterThrehold)].sum(axis=1)
    ew_fs = rd[(rd>0)&(abs(rd)>filterThrehold)].sum(axis=1)
    return pd.concat([ew_absorb, ew_fs, specLines], axis=1)

def smooth(filterSet):
    n = 5  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    filterSet = lfilter(b, a, filterSet, axis=1)
    return pd.DataFrame(filterSet)

def EWByRangeList(d, rangeList, filterThrehold=1, isSmooth=False, doScale=True):
    if doScale:
        d = pd.DataFrame(scale(d, axis=1))
    if isSmooth:
        d=smooth(d)
    ewfAll = pd.DataFrame()
    for r in rangeList:
        start, end = r
        ewf = intervalEWFeature(d, start, end, filterThrehold)
        newNames = ['ew_' + str(start) + '_' + str(end) + '_' + str(v) for v in ewf.columns.values]
        ewf.columns = newNames
        ewfAll = pd.concat([ewfAll,ewf], axis=1)
    return ewfAll

In [5]:
def submit(fileName, testTotal, preds):
    submission = testTotal[['id']]
    submission['pred'] = preds
    submission.to_csv('./predicts/'+fileName, header=False, index=False)
    return submission
    

In [6]:
def extractFeature(train_feature, partition=2, randomPartion=False, prefix="p", onlyAvg=False, filterLow=False, levelFeature=False, scaleSpec=False, smooth=False):
    if scaleSpec:
        train_feature = pd.DataFrame(scale(train_feature, axis=1))
    if smooth:
        train_feature = train_feature.rolling(100, axis=1).mean()
    if filterLow:
        train_feature[train_feature<1] = 0
    plen = train_feature.shape[1]/partition
    features = pd.DataFrame()
    avgs = pd.DataFrame()
    for i in range(0, partition):
        pstart = random.randint(0, train_feature.shape[1]-plen-1)
        pendExclue = pstart + plen + 1
        avgC = train_feature.iloc[:, pstart:pendExclue].mean(axis=1)
        stdC = train_feature.iloc[:, pstart:pendExclue].std(axis=1)
        maxC = train_feature.iloc[:, pstart:pendExclue].max(axis=1)
        minC = train_feature.iloc[:, pstart:pendExclue].min(axis=1)
        medianC = train_feature.iloc[:, pstart:pendExclue].median(axis=1)
        diffC = train_feature.iloc[:, pstart:pendExclue].diff(axis=1).iloc[:,1:]
        features[prefix+'_avg'+str(i)] = avgC
        avgs[prefix+'_avg'+str(i)] = avgC
        if onlyAvg==False:
            features[prefix+'_std'+str(i)] = stdC
            features[prefix+'_median'+str(i)] = medianC
            features[prefix+'_max'+str(i)] = maxC
            features[prefix+'_min'+str(i)] = minC
            features[prefix+'_avg_m_std'+str(i)] = stdC/avgC
            features[prefix+'_max_m_avg'+str(i)] = maxC/avgC
            features[prefix+'_min_m_avg'+str(i)] = minC/avgC
#             features[prefix+'_media_m_avg'+str(i)] = medianC/avgC
    if levelFeature:
        level_feature = extractFeature(avgs, prefix="l", partition = 4)
        features = pd.concat([features, level_feature], axis = 1)
    return features.fillna(0)

In [7]:
def readTotalFeature(fileName, maxFileIndex, hasLabel=True, partition = 20, ewFeature=True, addQsoFeature=True):
    total_train_feature = pd.DataFrame()
    for i in range(0, maxFileIndex+1):
        print('processing:' + str(i))
        train_set = pd.read_csv(fileName + str(i) +'.csv', header=None)
        renameCol = {0:'id'}
        if hasLabel:
            renameCol[train_set.shape[1]-1] = 'label'
        train_set.rename(columns=renameCol, inplace=True)
        if hasLabel:
            processed_train = pd.DataFrame()
            rawFeature = train_set.iloc[:,1:-1]
        else:
            rawFeature = train_set.iloc[:,1:]
        processed_train = extractFeature(rawFeature, partition, levelFeature=False, smooth=False)
        if ewFeature:
            rangeList = [(0,300), (0, 500), (500, 1000),(850, 1000), (1000, 1300),(1500,2000), (1500, 2600), (2500, 2600)]
            ew_feature = EWByRangeList(rawFeature, rangeList, filterThrehold=0.5, doScale=True)
            processed_train = pd.concat([processed_train, ew_feature], axis=1)
        if addQsoFeature:
            qso_feature = qsoFeature(rawFeature)
            processed_train = pd.concat([processed_train, qso_feature], axis=1)
        if hasLabel:
            processed_train['label'] = train_set.label
        else:
            processed_train['id'] = train_set.id
        total_train_feature = total_train_feature.append(processed_train, ignore_index=True)
    return total_train_feature.sample(total_train_feature.shape[0], replace=False)

In [25]:
validMode = True
reFeature = True
partition = 40
rng = np.random.RandomState(42)

In [15]:
if reFeature:
    total_train_feature = readTotalFeature('./train_feature_all_', 48, partition=partition)
    total_train_feature.to_csv('./train_feature_check_point.csv', index=False)
else :
    total_train_feature = pd.read_csv('./train_feature_check_point.csv')

processing:0
processing:1
processing:2
processing:3
processing:4
processing:5
processing:6
processing:7
processing:8
processing:9
processing:10
processing:11
processing:12
processing:13
processing:14
processing:15
processing:16
processing:17
processing:18
processing:19
processing:20
processing:21
processing:22
processing:23
processing:24
processing:25
processing:26
processing:27
processing:28
processing:29
processing:30
processing:31
processing:32
processing:33
processing:34
processing:35
processing:36
processing:37
processing:38
processing:39
processing:40
processing:41
processing:42
processing:43
processing:44
processing:45
processing:46
processing:47
processing:48


In [16]:
total_train_feature.head()

,p_avg0,p_std0,p_median0,p_max0,p_min0,p_avg_m_std0,p_max_m_avg0,p_min_m_avg0,p_avg1,p_std1,...,ew_1500_2600_fs,ew_1500_2600_ss,ew_2500_2600_0,ew_2500_2600_1,ew_2500_2600_fs,ew_2500_2600_ss,qso_f1,qso_f2,qso_f3,label
310881,3796.510988,54.985870,3801.186639,3916.869977,3660.682282,0.014483,1.031703,0.964223,4588.783073,266.289995,...,0,0,-88.814748,0.000000,0,0,False,True,1.373977,star
57788,197.380327,9.553809,197.801202,220.833376,168.192388,0.048403,1.118822,0.852123,149.978671,13.542624,...,0,1,-86.558126,0.000000,0,0,False,True,1.538935,star
371916,11312.741618,245.421672,11360.894738,11708.643895,10559.634596,0.021694,1.034996,0.933428,10873.911141,132.805000,...,0,0,-143.123892,0.000000,0,0,False,True,1.250234,star
358564,281.821613,9.341247,281.998455,300.606610,245.134547,0.033146,1.066656,0.869822,308.920589,6.577458,...,0,0,0.000000,0.000000,0,0,False,True,7.998486,star
456123,179.415320,9.617548,181.508454,196.059274,140.201435,0.053605,1.092768,0.781435,192.935475,5.812845,...,0,0,0.000000,0.507417,0,0,False,True,2.001043,star


In [17]:
total_train_feature = total_train_feature.fillna(0)

In [18]:
total_train_feature[total_train_feature.label=='unknown'].head(1000)

,p_avg0,p_std0,p_median0,p_max0,p_min0,p_avg_m_std0,p_max_m_avg0,p_min_m_avg0,p_avg1,p_std1,...,ew_1500_2600_fs,ew_1500_2600_ss,ew_2500_2600_0,ew_2500_2600_1,ew_2500_2600_fs,ew_2500_2600_ss,qso_f1,qso_f2,qso_f3,label
260750,217.138517,35.930692,217.251443,348.529747,150.217915,0.165474,1.605103,0.691807,301.352013,55.001680,...,6,12,-16.696785,11.276247,1,4,False,False,0.945924,unknown
140296,32.626835,4.920498,31.612714,47.050526,19.375413,0.150811,1.442081,0.593849,31.562800,7.872444,...,11,4,-1.301078,53.278792,3,2,False,True,9.242844,unknown
150154,2.643602,1.037926,2.784521,5.011195,0.038429,0.392618,1.895594,0.014537,4.464232,0.740493,...,6,9,0.000000,74.083319,0,1,False,True,3.058971,unknown
37038,5.583741,15.980089,4.488848,45.533075,-47.388274,2.861897,8.154583,-8.486833,20.969943,2.980366,...,19,45,-85.110020,11.587920,5,13,False,True,12.168809,unknown
337616,9.150031,0.884372,9.142968,11.271781,6.575082,0.096652,1.231884,0.718586,9.717124,2.443043,...,7,15,-107.620554,1.812273,1,6,False,True,29.221362,unknown
149635,18.519849,6.162847,19.389905,37.038299,-4.258222,0.332770,1.999924,-0.229927,24.719582,14.598091,...,41,40,-21.857111,127.782609,18,12,False,True,1.703672,unknown
173368,24.014913,2.937416,23.781886,36.897230,11.021559,0.122316,1.536430,0.458946,21.500097,2.146870,...,11,5,-3.211552,84.666074,8,3,False,True,5.881546,unknown
48468,0.446473,0.335523,0.442285,1.675456,-0.735220,0.751498,3.752651,-1.646731,0.711764,0.251776,...,0,9,-76.843109,0.000000,0,3,False,True,3.898536,unknown
164835,81.851335,2.646111,81.873570,86.910172,73.321247,0.032328,1.061805,0.895786,85.707601,3.303976,...,0,0,0.000000,24.366475,0,0,False,False,0.669850,unknown
308764,473.755064,3269.449765,82.088204,26631.293683,0.000000,6.901139,56.213212,0.000000,65.678602,19.351708,...,0,0,0.000000,0.574542,0,0,False,True,0.009534,unknown


In [19]:
X = total_train_feature.iloc[:,:-1]
Y = total_train_feature.iloc[:,-1]

# Split the dataset in two equal parts
x, t_x, y, t_y = train_test_split(
    X, Y, test_size=0.3, random_state=0)
x.shape


(338695, 355)

In [20]:
def outlier_rejection(X, y):
    model = IsolationForest(n_jobs=-1, random_state=rng)
    model.fit(X)
    y_pred = model.predict(X)
    return X[y_pred == 1], y[y_pred == 1]

In [1]:
smote = SMOTE()
anova_filter = SelectKBest(f_classif, k=(int)(x.shape[1]*0.9))
model = RandomForestClassifier(n_jobs=8, class_weight='balanced', max_depth=60, n_estimators=100, bootstarp=False,  verbose=1,random_state=rng)
# model = AdaBoostClassifier(random_state=rng)
# model = GradientBoostingClassifier(random_state=rng,verbose=1)
# model = XGBClassifier()
# model = KNeighborsClassifier(n_neighbors = 3, n_jobs=-1)
scaler = StandardScaler()
clf = ImPineline([
    ('anova', anova_filter),
    ('scaler',scaler),
    ('smote',smote),
    ('model', model)
])


NameError: name 'SMOTE' is not defined

In [ ]:
if validMode:
    rs = clf.fit(x, y)
    preds_test = clf.predict(t_x)
#     preds_test = carbateQso(t_x, preds_test)
#     clf.score(t_x, t_y)
    print("%0.3f" % f1_score(t_y, preds_test, average='macro'))  
    print(classification_report(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))
    print(confusion_matrix(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))

In [ ]:
t_x[t_y!=preds_test].index

In [114]:
def carbateQso(t_x, preds_test):
    fPreds = pd.concat([t_x.reset_index(), pd.DataFrame(preds_test)], axis=1)
    fPreds[(fPreds.iloc[:,-1]=='unknown')&(fPreds.qso_f3>2)].iloc[:,-1] = 'qso'
    return fPreds.iloc[:,-1].values

In [120]:
if validMode:
    clf = RandomForestClassifier(n_jobs=-1, class_weight='balanced',  verbose=1 )
    rs = clf.fit(x, y)
    preds_test = clf.predict(t_x)
    # clf.score(t_x, t_y)
    print('f1: %.2f' % f1_score(t_y, preds_test, average='macro')) 
    print(classification_report(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))
    print(confusion_matrix(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))

[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.5s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.8s finished
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


f1: 0.44
             precision    recall  f1-score   support

     galaxy       0.71      0.09      0.16       113
        qso       0.67      0.07      0.13        28
       star       0.94      0.99      0.96      8191
    unknown       0.67      0.41      0.51       668

avg / total       0.92      0.93      0.92      9000

[[  10    0   83   20]
 [   0    2   13   13]
 [   2    1 8088  100]
 [   2    0  394  272]]


In [123]:
if validMode:
    rs = clf.fit(x, y)
    scores = cross_val_score(clf, x, y, cv=3, n_jobs=-1, verbose=1, scoring='f1_macro')
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#     preds_test = clf.predict(t_x)
    # clf.score(t_x, t_y)
#     print("%0.3f" % f1_score(t_y, preds_test, average='macro'))  

[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    9.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   12.8s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   13.2s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   13.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


Accuracy: 0.50 (+/- 0.03)


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   17.9s finished


In [32]:
if validMode:
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])
    plt.show()


In [33]:
if validMode:
    smote = SMOTE()
    parameters = {
        'max_depth':[60, 70, 80],  #70
        'max_features':[0.5,0.6,0.7], #0.6
    #     'min_samples_split':[2, 4, 8, 20], #2
    #     'min_samples_leaf':[1,4,8,20], #1
    #     'min_weight_fraction_leaf':[0,0.05,0.1,0.2], #0
        'max_leaf_nodes':[800, 1000, 1200], #1000
    #      'bootstrap':[True, False], #False
    #     'oob_score':[True, False], #False
    #     'class_weight':[None,'balanced'], #balanced
        'n_estimators':[10, 20], #10
    #     'criterion':['gini','entropy']
    }
    clf = GridSearchCV(RandomForestClassifier(n_jobs=4, max_depth=20), parameters, cv=5,
                           scoring='f1_macro', n_jobs=8, verbose=2)
    clf.fit(x, y)
    print(clf.best_params_)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [22]:
if reFeature:
    testTotal = readTotalFeature('./test_feature_all_', 9, hasLabel=False, partition=partition)
    testTotal.to_csv('./test_feature_checkpoint.csv', index=False)
    testTotal.head()
else :
    testTotal = pd.read_csv('./test_feature_checkpoint.csv')

processing:0
processing:1
processing:2
processing:3
processing:4
processing:5
processing:6
processing:7
processing:8
processing:9


In [49]:
testTotal.shape

(100000, 356)

In [50]:
X.shape

(483851, 355)

In [23]:
clf.fit(X, Y)



[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 12.4min
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed: 31.3min finished


Pipeline(memory=None,
     steps=[('anova', SelectKBest(k=319, score_func=<function f_classif at 0x1a17b11668>)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('smote', SMOTE(k=None, k_neighbors=5, kind='regular', m=None, m_neighbors=10, n_jobs=1,
   out_step=0.5, random_state=None, ratio='auto', svm_...random_state=<mtrand.RandomState object at 0x1c17eca8c0>,
            verbose=1, warm_start=False))])

In [24]:
test_x = testTotal.iloc[:,:-1]
pred = clf.predict(test_x)
timestr = time.strftime("%m%d_%H%M")
rs = submit(timestr+'_rf_ew_40.csv',testTotal, pred)


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.9s finished
/Users/cql/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [54]:
rs.pred.value_counts()

star       86850
unknown    10856
galaxy      1941
qso          353
Name: pred, dtype: int64

In [ ]:
total_train_feature.head()

In [ ]:
total_train_feature[:,:-1]